Copyright (c) 2023 [윤기태]

https://github.com/yoonkt200/python-data-analysis

[MIT License](https://github.com/yoonkt200/python-data-analysis/blob/master/LICENSE.txt)

# (가제) 파이썬 데이터 분석

-----

# <이것이 데이터 분석이다 with 파이썬> 4쇄 이후 변경된 크롤링 파트
- 네이버 뉴스 헤드라인 데이터 크롤링

### 바로가기

- [<Step1. 크롤링> : 크롤링으로 웹 데이터 가져오기](#<Step1.-크롤링>-:-크롤링으로-웹-데이터-가져오기)
    - [BeautifulSoup을 이용한 웹 크롤링]
    - [네이버 뉴스 헤드라인 데이터 크롤링]

-----

- 맥 OS, 리눅스 OS의 경우 /{Local Path}/anaconda3/envs/{env name}/lib/python3.{xx}/site-packages/pytagcloud/fonts의 경로에 위와 동일한 방법을 적용해줍니다.

In [5]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# <Step1. 크롤링> : 크롤링으로 웹 데이터 가져오기

### [BeautifulSoup을 이용한 웹 크롤링]

- 아래 코드 실행을 위해, anaconda prompt 혹은 Terminal에서 아래와 같은 패키지들을 설치해 줍니다.
    - (env_name) `pip install selenium`
    - (env_name) `pip install beautifulsoup4`
- 혹은 아래의 코드로 라이브러리를 설치합니다.

In [6]:
!pip install selenium beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached exceptiongroup-1.1.0-py3-none-any.whl (14 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


-----
- 책에 기재된 나무위키 페이지 대신, 네이버 뉴스 홈의 특정 섹터를 크롤링하는 예제로 변경하였습니다.
- 아래 이미지는 '정치' 섹터의 헤드라인 뉴스 크롤링을 하기 위한 페이지입니다.

![앱 생성 페이지](img/new_crawl_1.png)

##### 페이지 리스트 가져오기 (헤드라인 뉴스들의 링크 가져오기)

In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re

# brew 로 설치된 chromedriver의 path (Mac)
mac_path = "/usr/local/bin/chromedriver"  # m1 이전 맥 OS
# mac_path = "/opt/homebrew/bin/chromedriver"  # m1 이후 맥 OS

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows)
window_path = "chromedriver.exe"

# 크롤링할 사이트 주소를 입력합니다.
source_url = "https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100"

# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
driver = webdriver.Chrome(mac_path)  # for Mac
# driver = webdriver.Chrome(window_path)  # for Windows
driver.get(source_url)
req = driver.page_source
soup = BeautifulSoup(req, "html.parser")
cluster_text = soup.find_all(name="div", attrs={"class":"cluster_text"})

In [8]:
# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_urls = []
for index in range(0, len(cluster_text)):
    cluster = cluster_text[index]
    news_url = cluster.find(name="a", attrs={"class":"cluster_text_headline nclicks(cls_pol.clsart)"})
    if news_url is not None:
        page_urls.append(news_url.get("href"))

# 중복 url을 제거합니다.
page_urls = list(set(page_urls))

# 다섯 개의 페이지를 출력합니다.
for page in page_urls[:5]:
    print(page)

# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

https://n.news.naver.com/mnews/article/417/0000899014?sid=100
https://n.news.naver.com/mnews/article/422/0000586143?sid=100
https://n.news.naver.com/mnews/article/008/0004855984?sid=100
https://n.news.naver.com/mnews/article/020/0003482210?sid=100
https://n.news.naver.com/mnews/article/421/0006653606?sid=100


-----

![앱 생성 페이지](img/new_crawl_2.png)

##### 페이지내 텍스트 구조 확인

In [9]:
driver = webdriver.Chrome(mac_path)  # for Mac
# driver = webdriver.Chrome(window_path)  # for Windows
driver.get(page_urls[0])
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')
title_area = soup.find(name="div", attrs={"class":"media_end_head_title"})
title = title_area.find_all('h2')[0]
content_paragraphs = soup.find(name="div", attrs={"class":"newsct_article"})
content_corpus = content_paragraphs.text

print(title.text)
print("\n")
print(content_corpus)

# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

"28석만"… 與, 이재명 체포동의안 관련 '이탈표' 촉구








국민의힘이 이재명 더불어민주당 대표에 대한 국회 체포동의안과 관련해 가결 쪽에 힘을 실으며 야당 측 이탈표를 호소하는 모양새다. 사진은 지난 24일 서울 여의도 국회에서 열린 제403회국회(임시회) 제7차 본회의에서 이 대표 체포동의안 및 기타 법안이 보고되는 모습. /사진=장동규 기자 국민의힘이 표결을 앞둔 이재명 더불어민주당 대표에 대한 체포동의안과 관련해 야당을 압박하며 막판 이탈표 호소에 나섰다. 국회는 이날 오후 2시30분 본회의를 열고 국회법에 따라 이 대표에 대한 체포동의안 표결에 들어간다. 국회는 지난 20일 특정경제범죄가중처벌등에관한법률(배임) 위반 혐의 등을 받는 이 대표에 대한 체포동의안을 접수했다.국회는 체포동의안을 접수하면 국회의장이 본회의에 보고하고 24시간 이후 72시간 이내에 무기명 투표를 진행한다. 이에 이 대표에 대한 체포동의안은 지난 24일 국회 본회의에서 보고된 뒤 이날 표결에 부쳐질 예정이다.체포동의안이 가결되려면 재적 의원 과반 출석에 과반 찬성이 필요하다. 국회 전체 의석 299석 중 민주당은 169석, 국민의힘 115석, 정의당 6석, 기본소득당 1석, 시대전환 1석, 무소속 7석 등이다. 민주당이 다수 의석을 가진 만큼 이 대표의 체포동의안 부결 가능성이 높게 점쳐지지만 민주당 안에서 28석 이상의 이탈표가 발생할 경우 체포동의안이 가결될 수 있다.이에 국민의힘은 비상대책위원회의와 원내대책회의·의원총회 등에서 체포동의안 가결과 이 대표 구속을 주장하며 '이재명·민주당 때리기'에 집중했다. 국회 의석 과반 이상인 민주당의 단합에 부결 가능성을 높게 점치고 있으면서도 결과와 관계없이 '방탄 국회' 이미지를 강조하며 여론의 우위를 선점한다는 계획이다.주호영 원내대표는 연일 이 대표의 체포동의안을 겨냥해 수위 높은 비판을 내뱉었다. 주 원내대표는 "이 대표가 여러가지 부정부패 혐의를 받는 것은 민주당 뿐 아니라 국회 전체의 위신을 떨어뜨리고 있다"며 

-----

### [네이버 뉴스 헤드라인 데이터 크롤링]

In [10]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비합니다.
columns = ["title", "content_text"]
df = pd.DataFrame(columns=columns)

# 각 페이지별 '제목', '본문' 정보를 데이터 프레임으로 만듭니다.
for page_url in page_urls:

    # 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
    driver = webdriver.Chrome(mac_path)  # for Mac
    # driver = webdriver.Chrome(window_path)  # for Windows
    driver.get(page_url)
    req = driver.page_source
    soup = BeautifulSoup(req, "html.parser")
    
    title_area = soup.find(name="div", attrs={"class":"media_end_head_title"})
    title = title_area.find_all('h2')[0]
    content_paragraphs = soup.find(name="div", attrs={"class":"newsct_article"})
    content_corpus = content_paragraphs.text
    
    # 제목 정보에서 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""
    
    
    # 모든 정보를 하나의 데이터 프레임에 저장합니다.
    row = [row_title, content_corpus]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)
    
    # 크롤링에 사용한 브라우저를 종료합니다.
    driver.close()

In [11]:
# 데이터 프레임을 출력합니다.
df.head(3)

,title,content_text
0,"""28석만""… 與, 이재명 체포동의안 관련 '이탈표' 촉구",\n\n\n\n\n\n국민의힘이 이재명 더불어민주당 대표에 대한 국회 체포동의안과 ...
1,"[속보] 윤대통령 ""학폭 근절 대책 조속히 보고"" 지시",\n\n\t\t\t#윤대통령 #수석비서관회의 #교육부 #학교폭력 #근절대책연합뉴스T...
2,"與 최고위원 후보 8인, 첫 토론회…""총선승리 앞장서겠다""",\n\n\n\n\n\n(서울=뉴스1) 이재명 기자 = 국민의힘 최고위원 후보들이 2...


-----

## 크롤링 이후의 분석 내용은 책에 기재되어 있는 내용인 `01-namu-wiki-text-analysis.ipynb` 파일과 동일합니다.